In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests

In [2]:
# URL of page to be scraped
url = 'https://www.qsrmagazine.com/content/ranking-top-50-fast-food-chains-america'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-2fca3af1d16ddfeabea",applicationID:"556389158"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var i=n[t]={exports:{}};e[t][0].call(i.exports,function(n){var i=e[t][1][n];return r(i||n)},i,i.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(e,n,t){function r(){}function i(e,n,t){return function(){return o(e,[u.now()].concat(f(arg

In [6]:
chart = soup.find(id="block-qsr2020-content")
data_company = [dc.get_text() for dc in chart.select(".sortable-chart-container .data-company")]
data_category = [dct.get_text() for dct in chart.select(".sortable-chart-container .data-category")]
all_sales = [als.get_text() for als in chart.select(".sortable-chart-container .data-2018_us_systemwide_sales_millions")]
avg_sales = [avs.get_text() for avs in chart.select(".sortable-chart-container .data-2018_average_sales_per_unit_thousands")]
franchise_units = [fu.get_text() for fu in chart.select(".sortable-chart-container .data-2018_franchiselicense_units")]
company_units = [cu.get_text() for cu in chart.select(".sortable-chart-container .data-2018_company_units")]
total_units = [tu.get_text() for tu in chart.select(".sortable-chart-container .data-2018_total_units")]

In [7]:
import pandas as pd
restaurants = pd.DataFrame({
    "Company": data_company,
    "Category": data_category,
    "Systemwide_Sales_Millions": all_sales,
    "Avg_Sales_per_Unit_Thousands": avg_sales,
    "Franchise_Units": franchise_units,
    "Company_Units": company_units,
    "Total_Units": total_units
})

In [8]:
restaurants.head()

Company  Category Systemwide_Sales_Millions  \
0    McDonald's    Burger                 38,524.05   
1    Starbucks*     Snack                 19,700.00   
2       Subway*  Sandwich                 10,410.34   
3     Taco Bell    Global                 10,300.00   
4  Chick-fil-A*   Chicken                 10,000.00   

  Avg_Sales_per_Unit_Thousands Franchise_Units Company_Units Total_Units  
0                     2,768.73          13,229           685      13,914  
1                     1,349.32           6,332         8,493      14,825  
2                       420.00          24,798             0      24,798  
3                     1,600.00           6,126           462       6,588  
4                     4,166.67           2,400             0       2,400

In [9]:
# transform data by setting the index
transformed_restaurants = restaurants.copy()

In [10]:
transformed_restaurants.head()

Company  Category Systemwide_Sales_Millions  \
0    McDonald's    Burger                 38,524.05   
1    Starbucks*     Snack                 19,700.00   
2       Subway*  Sandwich                 10,410.34   
3     Taco Bell    Global                 10,300.00   
4  Chick-fil-A*   Chicken                 10,000.00   

  Avg_Sales_per_Unit_Thousands Franchise_Units Company_Units Total_Units  
0                     2,768.73          13,229           685      13,914  
1                     1,349.32           6,332         8,493      14,825  
2                       420.00          24,798             0      24,798  
3                     1,600.00           6,126           462       6,588  
4                     4,166.67           2,400             0       2,400

In [11]:
transformed_restaurants["Company"].is_unique

True

In [12]:
#transformed_restaurants.set_index("Company", inplace=True)

In [13]:
transformed_restaurants.head()

Company  Category Systemwide_Sales_Millions  \
0    McDonald's    Burger                 38,524.05   
1    Starbucks*     Snack                 19,700.00   
2       Subway*  Sandwich                 10,410.34   
3     Taco Bell    Global                 10,300.00   
4  Chick-fil-A*   Chicken                 10,000.00   

  Avg_Sales_per_Unit_Thousands Franchise_Units Company_Units Total_Units  
0                     2,768.73          13,229           685      13,914  
1                     1,349.32           6,332         8,493      14,825  
2                       420.00          24,798             0      24,798  
3                     1,600.00           6,126           462       6,588  
4                     4,166.67           2,400             0       2,400

In [14]:
transformed_restaurants = transformed_restaurants.assign(id=(transformed_restaurants['Company']).astype('category').cat.codes)
# transformed_restaurants                                        

In [15]:
transformed_restaurants.get_dtype_counts()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `get_dtype_counts` has been deprecated and will be removed in a future version. For DataFrames use `.dtypes.value_counts()
  """Entry point for launching an IPython kernel.


object    7
int8      1
dtype: int64

In [16]:
transformed_restaurants.loc["McDonald's" : , "Systemwide_Sales_Millions"]

Series([], Name: Systemwide_Sales_Millions, dtype: object)

In [17]:
extr = transformed_restaurants["Systemwide_Sales_Millions"]
type(extr) is tuple

False

In [18]:
transformed_restaurants["Company"] = transformed_restaurants["Company"].str.strip()
transformed_restaurants["Category"] = transformed_restaurants["Category"].str.strip()
transformed_restaurants["Systemwide_Sales_Millions"] = transformed_restaurants["Systemwide_Sales_Millions"].str.strip()
transformed_restaurants["Avg_Sales_per_Unit_Thousands"] = transformed_restaurants["Avg_Sales_per_Unit_Thousands"].str.strip()
transformed_restaurants["Franchise_Units"] = transformed_restaurants["Franchise_Units"].str.strip()
transformed_restaurants["Company_Units"] = transformed_restaurants["Company_Units"].str.strip()
transformed_restaurants["Total_Units"] = transformed_restaurants["Total_Units"].str.strip()
transformed_restaurants.head()

Company  Category Systemwide_Sales_Millions  \
0    McDonald's    Burger                 38,524.05   
1    Starbucks*     Snack                 19,700.00   
2       Subway*  Sandwich                 10,410.34   
3     Taco Bell    Global                 10,300.00   
4  Chick-fil-A*   Chicken                 10,000.00   

  Avg_Sales_per_Unit_Thousands Franchise_Units Company_Units Total_Units  id  
0                     2,768.73          13,229           685      13,914  29  
1                     1,349.32           6,332         8,493      14,825  40  
2                       420.00          24,798             0      24,798  42  
3                     1,600.00           6,126           462       6,588  43  
4                     4,166.67           2,400             0       2,400   8

In [19]:
transformed_restaurants["Systemwide_Sales_Millions"] = transformed_restaurants["Systemwide_Sales_Millions"].str.replace(',','').astype(float)
transformed_restaurants["Avg_Sales_per_Unit_Thousands"] = transformed_restaurants["Avg_Sales_per_Unit_Thousands"].str.replace(',','').astype(float)
transformed_restaurants["Franchise_Units"] = transformed_restaurants["Franchise_Units"].str.replace(',','').astype(float)
transformed_restaurants["Company_Units"] = transformed_restaurants["Company_Units"].str.replace(',','').astype(float)
transformed_restaurants["Total_Units"] = transformed_restaurants["Total_Units"].str.replace(',','').astype(float)

# transformed_restaurants["Systemwide_Sales_Millions"] 


In [20]:
transformed_restaurants = transformed_restaurants.infer_objects()
transformed_restaurants.dtypes

Company                          object
Category                         object
Systemwide_Sales_Millions       float64
Avg_Sales_per_Unit_Thousands    float64
Franchise_Units                 float64
Company_Units                   float64
Total_Units                     float64
id                                 int8
dtype: object

In [21]:
import sqlite3

In [22]:
conn = sqlite3.connect('restaurants.sqlite')
c = conn.cursor()

In [23]:
c.execute('CREATE TABLE rest_table (Company text, Category text, Systemwide_Sales_Millions number, Avg_Sales_per_Unit_Thousands number, Franchise_Units number, Company_Units number, Total_Units number, [id] number PRIMARY KEY)')
conn.commit()

In [24]:
transformed_restaurants.to_sql('rest_table', conn, if_exists='replace', index=False)
                 

In [25]:
c.execute('''  
SELECT * FROM rest_table
          ''')

In [26]:
for row in c.fetchall():
    print (row)

("McDonald's", 'Burger', 38524.05, 2768.73, 13229.0, 685.0, 13914.0, 29)
('Starbucks*', 'Snack', 19700.0, 1349.32, 6332.0, 8493.0, 14825.0, 40)
('Subway*', 'Sandwich', 10410.34, 420.0, 24798.0, 0.0, 24798.0, 42)
('Taco Bell', 'Global', 10300.0, 1600.0, 6126.0, 462.0, 6588.0, 43)
('Chick-fil-A*', 'Chicken', 10000.0, 4166.67, 2400.0, 0.0, 2400.0, 8)
("Wendy's", 'Burger', 9993.7, 1636.4, 5825.0, 353.0, 6711.0, 45)
('Burger King*', 'Burger', 9950.0, 1357.0, 7278.0, 49.0, 7327.0, 5)
("Dunkin'", 'Snack', 8787.0, 933.0, 9419.0, 0.0, 9419.0, 15)
("Domino's*", 'Pizza', 6600.0, 1123.0, 5486.0, 390.0, 5876.0, 14)
('Panera Bread*', 'Sandwich', 5734.63, 2739.91, 1131.0, 963.0, 2093.0, 32)
('Pizza Hut', 'Pizza', 5500.0, 878.0, 7432.0, 24.0, 7456.0, 35)
('Chipotle', 'Global', 4800.0, 2000.0, 0.0, 2491.0, 2491.0, 9)
('Sonic Drive-In', 'Burger', 4447.1, 1250.0, 3427.0, 179.0, 3606.0, 39)
('KFC', 'Chicken', 4400.0, 1210.0, 4019.0, 55.0, 4074.0, 25)
("Arby's", 'Sandwich', 3886.9, 1177.0, 2197.0, 1132.0, 

In [27]:
# def trim_all_columns(df):
#     """
#     Trim whitespace from ends of each value across all series in dataframe
#     """
#     trim_strings = lambda x: x.strip() if isinstance(x, str) else x
#     return df.applymap(trim_strings)


# # simple example of trimming whitespace from data elements
# transformed_restaurants = trim_all_columns(transformed_restaurants)
# print(transformed_restaurants)

In [28]:
transformed_restaurants.head()

Company  Category  Systemwide_Sales_Millions  \
0    McDonald's    Burger                   38524.05   
1    Starbucks*     Snack                   19700.00   
2       Subway*  Sandwich                   10410.34   
3     Taco Bell    Global                   10300.00   
4  Chick-fil-A*   Chicken                   10000.00   

   Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
0                       2768.73          13229.0          685.0      13914.0   
1                       1349.32           6332.0         8493.0      14825.0   
2                        420.00          24798.0            0.0      24798.0   
3                       1600.00           6126.0          462.0       6588.0   
4                       4166.67           2400.0            0.0       2400.0   

   id  
0  29  
1  40  
2  42  
3  43  
4   8

In [29]:
# Creating dataframe that shows only Burger Category 
burgers = transformed_restaurants.loc[transformed_restaurants["Category"]=="Burger",]
top_burgers = burgers.iloc[:3]
top_burgers

Company Category  Systemwide_Sales_Millions  \
0    McDonald's   Burger                   38524.05   
5       Wendy's   Burger                    9993.70   
6  Burger King*   Burger                    9950.00   

   Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
0                       2768.73          13229.0          685.0      13914.0   
5                       1636.40           5825.0          353.0       6711.0   
6                       1357.00           7278.0           49.0       7327.0   

   id  
0  29  
5  45  
6   5

In [30]:
# Creating dataframe that shows only Chicken Category 
chicken = transformed_restaurants.loc[transformed_restaurants["Category"]=="Chicken",]
top_chicken = chicken.iloc[:3]
top_chicken

Company Category  Systemwide_Sales_Millions  \
4   Chick-fil-A*  Chicken                    10000.0   
13           KFC  Chicken                     4400.0   
19      Popeyes*  Chicken                     3325.0   

    Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
4                        4166.67           2400.0            0.0       2400.0   
13                       1210.00           4019.0           55.0       4074.0   
19                       1415.00           2327.0           41.0       2368.0   

    id  
4    8  
13  25  
19  36

In [31]:
# Creating dataframe that shows only Sandwich Category 
Sandwich = transformed_restaurants.loc[transformed_restaurants["Category"]=="Sandwich",]
top_Sandwich = Sandwich.iloc[:3]
top_Sandwich

Company  Category  Systemwide_Sales_Millions  \
2         Subway*  Sandwich                   10410.34   
9   Panera Bread*  Sandwich                    5734.63   
14         Arby's  Sandwich                    3886.90   

    Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
2                         420.00          24798.0            0.0      24798.0   
9                        2739.91           1131.0          963.0       2093.0   
14                       1177.00           2197.0         1132.0       3329.0   

    id  
2   42  
9   32  
14   0

In [32]:
# Creating dataframe that shows only Sandwich Category 
Pizza = transformed_restaurants.loc[transformed_restaurants["Category"]=="Pizza",]
top_Pizza = Pizza.iloc[:3]
top_Pizza

Company Category  Systemwide_Sales_Millions  \
8         Domino's*    Pizza                     6600.0   
10        Pizza Hut    Pizza                     5500.0   
15  Little Caesars*    Pizza                     3800.0   

    Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
8                         1123.0           5486.0          390.0       5876.0   
10                         878.0           7432.0           24.0       7456.0   
15                         874.0           3700.0          562.0       4262.0   

    id  
8   14  
10  35  
15  26

In [33]:
# Creating dataframe that shows only Sandwich Category 
Snack = transformed_restaurants.loc[transformed_restaurants["Category"]=="Snack",]
top_Snack = Snack.iloc[:3]
top_Snack

Company Category  Systemwide_Sales_Millions  \
1    Starbucks*    Snack                    19700.0   
7       Dunkin'    Snack                     8787.0   
16  Dairy Queen    Snack                     3650.0   

    Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
1                        1349.32           6332.0         8493.0      14825.0   
7                         933.00           9419.0            0.0       9419.0   
16                       1204.58           4404.0            2.0       4406.0   

    id  
1   40  
7   15  
16  12

In [34]:
# Creating dataframe that shows only Global Category 
Global = transformed_restaurants.loc[transformed_restaurants["Category"]=="Global",]
top_Global = Global.iloc[:3]
# top_Global

In [35]:
total_by_category = transformed_restaurants.groupby(["Category"])[["Systemwide_Sales_Millions"]].sum()
total_by_category = total_by_category.reset_index()
total_by_category

Category  Systemwide_Sales_Millions
0    Burger                   78902.46
1   Chicken                   25495.01
2    Global                   20887.33
3     Pizza                   20037.35
4  Sandwich                   26283.75
5     Snack                   33307.24

In [36]:
merged_top_df = pd.concat([top_burgers, top_chicken, top_Sandwich, top_Pizza, top_Global, top_Snack])
merged_top_df

Company  Category  Systemwide_Sales_Millions  \
0        McDonald's    Burger                   38524.05   
5           Wendy's    Burger                    9993.70   
6      Burger King*    Burger                    9950.00   
4      Chick-fil-A*   Chicken                   10000.00   
13              KFC   Chicken                    4400.00   
19         Popeyes*   Chicken                    3325.00   
2           Subway*  Sandwich                   10410.34   
9     Panera Bread*  Sandwich                    5734.63   
14           Arby's  Sandwich                    3886.90   
8         Domino's*     Pizza                    6600.00   
10        Pizza Hut     Pizza                    5500.00   
15  Little Caesars*     Pizza                    3800.00   
3         Taco Bell    Global                   10300.00   
11         Chipotle    Global                    4800.00   
18   Panda Express*    Global                    3400.00   
1        Starbucks*     Snack                   19700.00   
7           Dunkin'     Snack                    8787.00   
16      Dairy Queen     Snack                    3650.00   

    Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
0                        2768.73          13229.0          685.0      13914.0   
5                        1636.40           5825.0          353.0       6711.0   
6                        1357.00           7278.0           49.0       7327.0   
4                        4166.67           2400.0            0.0       2400.0   
13                       1210.00           4019.0           55.0       4074.0   
19                       1415.00           2327.0           41.0       2368.0   
2                         420.00          24798.0            0.0      24798.0   
9                        2739.91           1131.0          963.0       2093.0   
14                       1177.00           2197.0         1132.0       3329.0   
8                        1123.00           5486.0          390.0       5876.0   
10                        878.00           7432.0           24.0       7456.0   
15                        874.00           3700.0          562.0       4262.0   
3                        1600.00           6126.0          462.0       6588.0   
11                       2000.00              0.0         2491.0       2491.0   
18                       1616.00            125.0         1979.0       2104.0   
1                        1349.32           6332.0         8493.0      14825.0   
7                         933.00           9419.0            0.0       9419.0   
16                       1204.58           4404.0            2.0       4406.0   

    id  
0   29  
5   45  
6    5  
4    8  
13  25  
19  36  
2   42  
9   32  
14   0  
8   14  
10  35  
15  26  
3   43  
11   9  
18  31  
1   40  
7   15  
16  12

In [37]:
# Selecting the top company for each category
top_rest_per_category = merged_top_df.loc[(merged_top_df["Company"] == "McDonald's") | (merged_top_df["Company"] == "Chick-fil-A*") | (merged_top_df["Company"] == "Subway*") | (merged_top_df["Company"] == "Domino's*") | (merged_top_df["Company"] == "Taco Bell") | (merged_top_df["Company"] == "Starbucks*")]
top_rest_per_category

Company  Category  Systemwide_Sales_Millions  \
0    McDonald's    Burger                   38524.05   
4  Chick-fil-A*   Chicken                   10000.00   
2       Subway*  Sandwich                   10410.34   
8     Domino's*     Pizza                    6600.00   
3     Taco Bell    Global                   10300.00   
1    Starbucks*     Snack                   19700.00   

   Avg_Sales_per_Unit_Thousands  Franchise_Units  Company_Units  Total_Units  \
0                       2768.73          13229.0          685.0      13914.0   
4                       4166.67           2400.0            0.0       2400.0   
2                        420.00          24798.0            0.0      24798.0   
8                       1123.00           5486.0          390.0       5876.0   
3                       1600.00           6126.0          462.0       6588.0   
1                       1349.32           6332.0         8493.0      14825.0   

   id  
0  29  
4   8  
2  42  
8  14  
3  43  
1  40

In [38]:
c.execute('''CREATE TABLE top_rest ([Company] text, [Category] text, [Systemwide_Sales_Millions] number, [Avg_Sales_per_Unit_Thousands] number, [Franchise_Units] number, [Company_Units] number, [Total_Units] number, [id] INTEGER PRIMARY KEY)''')
conn.commit()

In [39]:
merged_top_df.to_sql('top_rest', conn, if_exists='replace', index=False)

In [40]:
c.execute('''  
SELECT * FROM top_rest
          ''')

In [41]:
for row in c.fetchall():
    print (row)

("McDonald's", 'Burger', 38524.05, 2768.73, 13229.0, 685.0, 13914.0, 29)
("Wendy's", 'Burger', 9993.7, 1636.4, 5825.0, 353.0, 6711.0, 45)
('Burger King*', 'Burger', 9950.0, 1357.0, 7278.0, 49.0, 7327.0, 5)
('Chick-fil-A*', 'Chicken', 10000.0, 4166.67, 2400.0, 0.0, 2400.0, 8)
('KFC', 'Chicken', 4400.0, 1210.0, 4019.0, 55.0, 4074.0, 25)
('Popeyes*', 'Chicken', 3325.0, 1415.0, 2327.0, 41.0, 2368.0, 36)
('Subway*', 'Sandwich', 10410.34, 420.0, 24798.0, 0.0, 24798.0, 42)
('Panera Bread*', 'Sandwich', 5734.63, 2739.91, 1131.0, 963.0, 2093.0, 32)
("Arby's", 'Sandwich', 3886.9, 1177.0, 2197.0, 1132.0, 3329.0, 0)
("Domino's*", 'Pizza', 6600.0, 1123.0, 5486.0, 390.0, 5876.0, 14)
('Pizza Hut', 'Pizza', 5500.0, 878.0, 7432.0, 24.0, 7456.0, 35)
('Little Caesars*', 'Pizza', 3800.0, 874.0, 3700.0, 562.0, 4262.0, 26)
('Taco Bell', 'Global', 10300.0, 1600.0, 6126.0, 462.0, 6588.0, 43)
('Chipotle', 'Global', 4800.0, 2000.0, 0.0, 2491.0, 2491.0, 9)
('Panda Express*', 'Global', 3400.0, 1616.0, 125.0, 1979

In [42]:
c.execute('CREATE TABLE total_by_category (Company text, Category text, Systemwide_Sales_Millions number, Avg_Sales_per_Unit_Thousands number, Franchise_Units number, Company_Units number, Total_Units number, [id] number PRIMARY KEY)')
conn.commit()

In [43]:
total_by_category.to_sql('total_by_category', conn, if_exists='replace', index=False)

In [44]:
c.execute('''  
SELECT * FROM total_by_category
          ''')

In [45]:
for row in c.fetchall():
    print (row)

('Burger', 78902.45999999999)
('Chicken', 25495.01)
('Global', 20887.33)
('Pizza', 20037.350000000002)
('Sandwich', 26283.750000000004)
('Snack', 33307.24)


In [46]:
c.execute('CREATE TABLE top_rest_per_category (Company text, Category text, Systemwide_Sales_Millions number, Avg_Sales_per_Unit_Thousands number, Franchise_Units number, Company_Units number, Total_Units number, [id] number PRIMARY KEY)')
conn.commit()

In [47]:
top_rest_per_category.to_sql('top_rest_per_category', conn, if_exists='replace', index=False)

In [48]:
c.execute('''  
SELECT * FROM top_rest_per_category
          ''')

In [49]:
for row in c.fetchall():
    print (row)

("McDonald's", 'Burger', 38524.05, 2768.73, 13229.0, 685.0, 13914.0, 29)
('Chick-fil-A*', 'Chicken', 10000.0, 4166.67, 2400.0, 0.0, 2400.0, 8)
('Subway*', 'Sandwich', 10410.34, 420.0, 24798.0, 0.0, 24798.0, 42)
("Domino's*", 'Pizza', 6600.0, 1123.0, 5486.0, 390.0, 5876.0, 14)
('Taco Bell', 'Global', 10300.0, 1600.0, 6126.0, 462.0, 6588.0, 43)
('Starbucks*', 'Snack', 19700.0, 1349.32, 6332.0, 8493.0, 14825.0, 40)
